# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합-모듈화**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * Python 코드 모듈화
        * 각 모듈 코드 및 모델, 데이터파일을 일관성 있게 정리
        * .py 파일 생성 ==> 라이브러리 로딩, 각 task를 위한 함수 생성


## **1.환경설정**

* 경로 설정

구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = '/content/drive/MyDrive/kt/projects_kt/project_6_2/6_2'

## 2.모듈 구성하기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from warnings import filterwarnings
FutureWarning
filterwarnings('ignore')

In [ ]:
%%writefile /content/drive/MyDrive/project6_2/emergency.py

import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch


# 0. load key file------------------
save_directory = path + "fine_tuned_bert_v2"
audio_path = path + 'audio/'
client = OpenAI()
file_names = [f for f in os.listdir(audio_path) if os.path.isfile(os.path.join(audio_path, f))]

# 1-1 audio2text--------------------
def audio_to_text(audio_path, filename):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 오디오 파일을 읽어서, 위스퍼를 사용한 변환
    audio_file = open(audio_path + filename, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        language="ko",
        response_format="text",
    )


    # 결과 반환

    return transcript


# 1-2 text2summary------------------

def text_summary(input_text):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 시스템 역할과 응답 형식 지정
    system_role = '''당신은 신문기사에서 핵심을 요약하는 어시스턴트입니다.
    응답은 다음의 형식을 지켜주세요
    {"summary": \"텍스트 요약\"}
    '''

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )

    # 응답 받기
    answer = response.choices[0].message.content


    # 응답형식을 정리하고 return
    return answer
udio2text_df = pd.DataFrame(columns=['filename', 'text'])
text_summary_df = pd.DataFrame(columns=['filename', 'summary'])
# 반복문 수행하면서 오디오 변환
for filename in file_names:
    text = audio_to_text(audio_path, filename)
    summary = text_summary(text)

    # 데이터프레임에 추가
    # audio2text_df = pd.concat([audio2text_df, pd.DataFrame({'filename': [filename], 'text': [text]})], ignore_index=True)
    text_summary_df = pd.concat([text_summary_df, pd.DataFrame({'filename': [filename], 'summary': [summary]})], ignore_index=True)
# 데이터프레임 결과 조회
print(text_summary_df)
# 2. model prediction------------------

model2 = AutoModelForSequenceClassification.from_pretrained(save_directory)
# 2_2. 토크나이저 로드
tokenizer2 = AutoTokenizer.from_pretrained(save_directory)

# 3-1. get_distance------------------
def get_dist(start_lat, start_lng, dest_lat, dest_lng):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": 'sb7vih3g35',
        "X-NCP-APIGW-API-KEY": '4hp1aMpvQ9mRmhQ6h9U1xd5hV96x2so6Iz74vYiU',
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, headers = headers , params = params)


    if response.status_code == 200:
        data = response.json()  # JSON 응답을 파싱
        try:
            dist = data['route']['trafast'][0]['summary']['distance']  # 거리(m)
            return dist
        except (KeyError, IndexError):
            raise ValueError("올바르지 않은 응답 데이터 형식입니다.")
    else:
        raise ConnectionError(f"API 요청 실패: {response.status_code}, {response.text}")


def recommend_hospital3(lat, lng, alpha=0.1):

    min_lat = lat - alpha
    max_lat = lat + alpha
    min_lng = lng - alpha
    max_lng = lng + alpha

    # 범위 내 응급실 필터링
    target_hospitals = data[(data['위도'] >= min_lat) & (data['위도'] <= max_lat) &
                           (data['경도'] >= min_lng) & (data['경도'] <= max_lng)]

    box = []
    current_location = (lat, lng)
    for i in target_hospitals.index:
        box.append({'name': target_hospitals['병원이름'][i],
                    'address': target_hospitals['주소'][i],
                    'distance': get_dist(current_location[0],current_location[1],
                                         data['위도'][i],data['경도'][i])})

    box.sort(key=lambda x: x['distance'])
    box = pd.DataFrame(box)
    return box.head(3)

# 데이터 예측 함수
def predict(text, model, tokenizer, device):
    # 입력 문장 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 입력 텐서를 동일한 디바이스로 이동

    # 모델을 지정된 디바이스로 이동
    model = model.to(device)

    # 모델 예측
    with torch.no_grad():
        outputs = model(**inputs)

    # 로짓을 소프트맥스로 변환하여 확률 계산
    logits = outputs.logits
    probabilities = logits.softmax(dim=1)

    # 가장 높은 확률을 가진 클래스 선택
    pred = torch.argmax(probabilities, dim=-1).item()

    return pred, probabilities

# 3-2. recommendation------------------
from geopy.geocoders import Nominatim

def get_lat_long(address):
    # Geolocator 초기화
    geolocator = Nominatim(user_agent="chiricuto")

    # 주소를 위도/경도로 변환
    location = geolocator.geocode(address)

    if location:
        return location.latitude, location.longitude
    else:
        return "위치 정보를 찾을 수 없습니다."

for filename in file_names:
    text = audio_to_text(audio_path, filename)
    summary = text_summary(text)

    # 데이터프레임에 추가
    # audio2text_df = pd.concat([audio2text_df, pd.DataFrame({'filename': [filename], 'text': [text]})], ignore_index=True)
    text_summary_df = pd.concat([text_summary_df, pd.DataFrame({'filename': [filename], 'summary': [summary]})], ignore_index=True)
# 데이터프레임 결과 조회
for i in range(len(text_summary_df)):
  text = text_summary_df['summary'][i]
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  predicted_class, probabilities = predict(text, model2, tokenizer2,device)

  if predicted_class > 2:
    print("응급환자 대상이 아닙니다.")
  else :
    print("응급환자 대상입니다.")
    address = input("주소를 입력해주세요 : ")
    lat, lng = get_lat_long(address)
    print(f"주소: {address}")
    print(f"위도: {lat}, 경도: {lng}")
    recommend_hospital3(lat, lng)
    print("응급실 추천이 완료되었습니다.")





In [ ]:
!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim

def get_lat_long(address):
    # Geolocator 초기화
    geolocator = Nominatim(user_agent="chiricuto")

    # 주소를 위도/경도로 변환
    location = geolocator.geocode(address)

    if location:
        return location.latitude, location.longitude
    else:
        return "위치 정보를 찾을 수 없습니다."

# 테스트 예시
address = "서울특별시 강남구 테헤란로"
latitude, longitude = get_lat_long(address)

print(f"주소: {address}")
print(f"위도: {latitude}, 경도: {longitude}")


주소: 서울특별시 강남구 테헤란로
위도: 37.505562556835514, 경도: 127.05286314702252


In [ ]:
!pip install polyline

In [ ]:
# %%writefile '/content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/emergency.py'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openai
from openai import OpenAI
import torch
from IPython.display import display, IFrame

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import os
import polyline
import folium

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from warnings import filterwarnings

filterwarnings('ignore')
def load_file(filepath):
      with open(filepath, 'r') as file:
        return file.readline().strip()
class EmergencyAssistant:
    def __init__(self, save_directory, audio_path, hospital_data):
        self.save_directory = save_directory
        self.audio_path = audio_path
        self.hospital_data = pd.read_csv(hospital_data)  # 병원 데이터 로드
        self.client = OpenAI()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = AutoModelForSequenceClassification.from_pretrained(save_directory).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(save_directory)


    def audio_to_text(self, filename):
        audio_file = open(os.path.join(self.audio_path, filename), "rb")
        transcript = self.client.audio.transcriptions.create(
            file=audio_file,
            model="whisper-1",
            language="ko",
            response_format="text",
        )
        return transcript

    def text_summary(self, input_text):
        system_role = '''당신은 신문기사에서 핵심을 요약하는 어시스턴트입니다.
        응답은 다음의 형식을 지켜주세요:
        {"summary": "텍스트 요약"}
        '''
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": input_text},
            ]
        )
        return response.choices[0].message.content

    def predict(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits.softmax(dim=1)
        predicted_class = torch.argmax(logits, dim=-1).item()
        return predicted_class, logits

    def get_lat_long(self, address):
        geolocator = Nominatim(user_agent="emergency_assistant")
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            raise ValueError("주소를 찾을 수 없습니다.")

    def get_dist(self, start_lat, start_lng, dest_lat, dest_lng):
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": 'sb7vih3g35',
            "X-NCP-APIGW-API-KEY": '4hp1aMpvQ9mRmhQ6h9U1xd5hV96x2so6Iz74vYiU',
        }
        params = {
            "start": f"{start_lng},{start_lat}",
            "goal": f"{dest_lng},{dest_lat}",
            "option": "trafast"
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            try:
                return data['route']['trafast'][0]['summary']['distance']
            except (KeyError, IndexError):
                raise ValueError("올바르지 않은 응답 데이터 형식입니다.")
        else:
            raise ConnectionError(f"API 요청 실패: {response.status_code}, {response.text}")

    def recommend_hospital(self, lat, lng, alpha=0.1):
        target_hospitals = self.hospital_data[
            (self.hospital_data['위도'] >= lat - alpha) & (self.hospital_data['위도'] <= lat + alpha) &
            (self.hospital_data['경도'] >= lng - alpha) & (self.hospital_data['경도'] <= lng + alpha)
        ]
        recommendations = []
        for _, hospital in target_hospitals.iterrows():
            distance = self.get_dist(lat, lng, hospital['위도'], hospital['경도'])
            recommendations.append({
                'name': hospital['병원이름'],
                'address': hospital['주소'],
                'distance': distance,
                '위도' : hospital['위도'],
                '경도' : hospital['경도']
            })
        recommendations.sort(key=lambda x: x['distance'])
        return pd.DataFrame(recommendations).head(3)

    def process_audio_files(self):
        file_names = [f for f in os.listdir(self.audio_path) if os.path.isfile(os.path.join(self.audio_path, f))]
        results = []
        for filename in file_names:
            text = self.audio_to_text(filename)
            summary = self.text_summary(text)
            predicted_class, _ = self.predict(summary)
            results.append((filename, text, summary, predicted_class))
        return results

    def handle_emergency(self, address):
        lat, lng = self.get_lat_long(address)
        recommendations = self.recommend_hospital(lat, lng)
        return recommendations

    def visualize_hospitals(self, user_coords, hospitals):
        m = folium.Map(location=user_coords, zoom_start=12)
        folium.Marker(location=user_coords, tooltip="사용자 위치", icon=folium.Icon(color="blue")).add_to(m)
        for _, row in hospitals.iterrows():
            folium.Marker(
                location=(row['위도'], row['경도']),
                tooltip=f"{row['name']} (거리: {row['distance']:.2f} m)",
                icon=folium.Icon(color="red")
            ).add_to(m)
        return m
    # 병원 지도 시각화 및 경로 표시 함수
    def visualize_hospitals_with_route(self, user_coords, hospitals):
        # 사용자 위치를 중심으로 지도를 생성
        m = folium.Map(location=user_coords, zoom_start=13)

        # 사용자 위치 마커 추가
        folium.Marker(
            location=user_coords,
            tooltip="사용자 위치",
            icon=folium.Icon(color="blue")
        ).add_to(m)

        # 병원 위치 마커 추가 (추천 병원들)
        for _, row in hospitals.iterrows():
            folium.Marker(
                location=(row['위도'], row['경도']),
                tooltip=f"{row['name']} (거리: {row['distance']:.2f} m)",
                icon=folium.Icon(color="red")
            ).add_to(m)

            # 경로 표시 추가
            start_lat, start_lng = user_coords
            dest_lat, dest_lng = row['위도'], row['경도']
            route = self.get_route(start_lat, start_lng, dest_lat, dest_lng)

            if route:
                # 경로 데이터를 폴리라인으로 디코딩하여 지도에 추가
                points = polyline.decode(route)
                folium.PolyLine(points, color="red", weight=5, opacity=0.7).add_to(m)

        return m

    def get_route(self, start_lat, start_lng, dest_lat, dest_lng):
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": 'sb7vih3g35',
            "X-NCP-APIGW-API-KEY": '4hp1aMpvQ9mRmhQ6h9U1xd5hV96x2so6Iz74vYiU',
        }
        params = {
            "start": f"{start_lng},{start_lat}",
            "goal": f"{dest_lng},{dest_lat}",
            "option": "trafast"
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            try:
                # 경로 데이터 가져오기
                route = data['route']['trafast'][0]['path']
                return polyline.encode(route)  # 경로를 폴리라인으로 인코딩
            except (KeyError, IndexError):
                raise ValueError("올바르지 않은 응답 데이터 형식입니다.")
        else:
            raise ConnectionError(f"API 요청 실패: {response.status_code}, {response.text}")
# Example usage
# if __name__ == "__main__":
#     save_directory = "/content/drive/MyDrive/project6_2/models/fine_tuned_bert_v2"
#     audio_path = "/content/drive/MyDrive/project6_2/audio/"
#     hospital_data = "/content/drive/MyDrive/project6_2/응급실 정보.csv.csv"

#     assistant = EmergencyAssistant(save_directory, audio_path, hospital_data)

#     # Audio 파일 처리
#     results = assistant.process_audio_files()
#     for filename, text, summary, predicted_class in results:
#         print(f"파일: {filename}, 요약: {summary}, 분류: {predicted_class}")

#         if predicted_class <= 2:  # 응급환자
#             address = input("주소를 입력해주세요: ")
#             recommendations = assistant.handle_emergency(address)
#             print("추천 병원:")
#             print(recommendations)



In [ ]:

path = '/content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/'
save_directory = path+ 'fine_tuned_bert_v2'
audio_path = path+"audio/"
hospital_data = path + "응급실 정보.csv"
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key
assistant = EmergencyAssistant(save_directory, audio_path, hospital_data)

# Audio 파일 처리
results = assistant.process_audio_files()
for filename, text, summary, predicted_class in results:
    print(f"파일: {filename}, 요약: {summary}, 분류: {predicted_class}")

    if predicted_class <= 2:  # 응급환자
        address = input("응급환자 입니다. 주소를 입력해주세요: ")

        recommendations = pd.DataFrame(assistant.handle_emergency(address))
        print("추천 병원:")
        for idx, row in recommendations.iterrows():
          print(f"-----------------------------------")
          print(f"병원 이름    : {row['name']}")
          print(f"주소        : {row['address']}")
          print(f"거리 (m)    : {row['distance']} m")
          print(f"-----------------------------------\n")

          # 병원 시각화
        lat, lng = assistant.get_lat_long(address)
        user_coords = (lat, lng)
        hospital_map = assistant.visualize_hospitals_with_route(user_coords, recommendations)
        # hospital_map.save("hospital_map.html")

        # 지도를 노트북에서 바로 보기
        display(hospital_map)  # 주피터 노트북 환경에서 지도를 바로 렌더링

        print("병원 위치가 포함된 지도를 화면에 표시합니다.")

    else:
        print("응급환자가 아닙니다.")



파일: audio1.mp3, 요약: {"summary": "네네가 계단에서 넘어져 머리에서 피가 나고 숨쉬고 있는 것으로 보이지만 어지럽다고 합니다. 응급실로 신속히 이동하는 것이 좋습니다. 상황을 진단하고 필요한 처치 및 검사를 받을 수 있도록 도와주세요."}, 분류: 0
응급환자 입니다. 주소를 입력해주세요: 송파구 올림픽공원
추천 병원:
-----------------------------------
병원 이름    : 성심의료재단강동성심병원
주소        : 서울특별시 강동구 성안로 150 (길동)
거리 (m)    : 2624 m
-----------------------------------

-----------------------------------
병원 이름    : 경찰병원
주소        : 서울특별시 송파구 송이로 123, 국립경찰병원 (가락동)
거리 (m)    : 3131 m
-----------------------------------

-----------------------------------
병원 이름    : 한국보훈복지의료공단중앙보훈병원
주소        : 서울특별시 강동구 진황도로61길 53 (둔촌동)
거리 (m)    : 3220 m
-----------------------------------



병원 위치가 포함된 지도를 화면에 표시합니다.
파일: audio4.mp3, 요약: {"summary": "머리를 박은 후에 어지러움과 메스꺼움을 느낀다면 즉시 의료진과 상담해야 합니다. 이러한 증상은 경련, 후유증, 뇌진탕 또는 다른 심각한 상황의 조짐일 수 있으므로 빠른 처치가 중요합니다."}, 분류: 0


KeyboardInterrupt: Interrupted by user

In [ ]:
%%writefile '/content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/emergency.py'

import os
import requests
import pandas as pd
import torch
import folium
import polyline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from geopy.geocoders import Nominatim
from openai import OpenAI
from warnings import filterwarnings

filterwarnings('ignore')


class EmergencyAssistant:
    def __init__(self, save_directory, audio_path, hospital_data):
        self.save_directory = save_directory
        self.audio_path = audio_path
        self.hospital_data = pd.read_csv(hospital_data)  # 병원 데이터 로드
        self.client = OpenAI()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = AutoModelForSequenceClassification.from_pretrained(save_directory).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(save_directory)

    def audio_to_text(self, filename):
        audio_file = open(os.path.join(self.audio_path, filename), "rb")
        transcript = self.client.audio.transcriptions.create(
            file=audio_file,
            model="whisper-1",
            language="ko",
            response_format="text",
        )
        return transcript

    def text_summary(self, input_text):
        system_role = '''당신은 신문기사에서 핵심을 요약하는 어시스턴트입니다.
        응답은 다음의 형식을 지켜주세요:
        {"summary": "텍스트 요약"}
        '''
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": input_text},
            ]
        )
        return response.choices[0].message.content

    def predict(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits.softmax(dim=1)
        predicted_class = torch.argmax(logits, dim=-1).item()
        return predicted_class, logits

    def get_lat_long(self, address):
        geolocator = Nominatim(user_agent="emergency_assistant")
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            raise ValueError("주소를 찾을 수 없습니다.")

    def get_dist(self, start_lat, start_lng, dest_lat, dest_lng):
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": 'sb7vih3g35',
            "X-NCP-APIGW-API-KEY": '4hp1aMpvQ9mRmhQ6h9U1xd5hV96x2so6Iz74vYiU',
        }
        params = {
            "start": f"{start_lng},{start_lat}",
            "goal": f"{dest_lng},{dest_lat}",
            "option": "trafast"
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            try:
                return data['route']['trafast'][0]['summary']['distance']
            except (KeyError, IndexError):
                raise ValueError("올바르지 않은 응답 데이터 형식입니다.")
        else:
            raise ConnectionError(f"API 요청 실패: {response.status_code}, {response.text}")

    def recommend_hospital(self, lat, lng, alpha=0.1):
        target_hospitals = self.hospital_data[
            (self.hospital_data['위도'] >= lat - alpha) & (self.hospital_data['위도'] <= lat + alpha) &
            (self.hospital_data['경도'] >= lng - alpha) & (self.hospital_data['경도'] <= lng + alpha)
        ]
        recommendations = []
        for _, hospital in target_hospitals.iterrows():
            distance = self.get_dist(lat, lng, hospital['위도'], hospital['경도'])
            recommendations.append({
                'name': hospital['병원이름'],
                'address': hospital['주소'],
                '위도': hospital['위도'],
                '경도': hospital['경도'],
                'distance': distance
            })
        recommendations.sort(key=lambda x: x['distance'])
        return pd.DataFrame(recommendations).head(3)

    def process_audio_files(self):
        file_names = [f for f in os.listdir(self.audio_path) if os.path.isfile(os.path.join(self.audio_path, f))]
        results = []
        for filename in file_names:
            text = self.audio_to_text(filename)
            summary = self.text_summary(text)
            predicted_class, _ = self.predict(summary)
            results.append((filename, text, summary, predicted_class))
        return results

    def handle_emergency(self, address):
        lat, lng = self.get_lat_long(address)
        recommendations = self.recommend_hospital(lat, lng)
        return recommendations

    # 병원 지도 시각화 및 경로 표시 함수
    # def visualize_hospitals_with_route(self, user_coords, hospitals):
    #   # 사용자 위치를 중심으로 지도를 생성
    #   m = folium.Map(location=user_coords, zoom_start=13)

    #   # 사용자 위치 마커 추가
    #   folium.Marker(
    #       location=user_coords,
    #       tooltip="사용자 위치",
    #       icon=folium.Icon(color="blue")
    #   ).add_to(m)

    #   # 병원 위치 마커 추가 (추천 병원들)
    #   for _, row in hospitals.iterrows():
    #       folium.Marker(
    #           location=(row['위도'], row['경도']),
    #           tooltip=f"{row['name']} (거리: {row['distance']:.2f} m)",
    #           icon=folium.Icon(color="red")
    #       ).add_to(m)

    #       # 경로 표시 추가
    #       try:
    #           # OSRM API를 사용하여 경로 데이터 가져오기
    #           url = f"http://router.project-osrm.org/route/v1/driving/{user_coords[1]},{user_coords[0]};{row['경도']},{row['위도']}?overview=full&geometries=geojson"
    #           response = requests.get(url)

    #           if response.status_code == 200:
    #               route_data = response.json()

    #               # GeoJSON 형식의 경로 데이터를 추출
    #               route_coordinates = route_data['routes'][0]['geometry']['coordinates']
    #               # 거리 정보 추출 (미터 단위)
    #               distance = route_data['routes'][0]['distance']

    #               # 거리를 km 단위로 변환
    #               distance_km = distance / 1000

    #               # 좌표 순서 변환 (경도,위도 -> 위도,경도)
    #               route_coordinates = [[coord[1], coord[0]] for coord in route_coordinates]

    #               # 경로를 지도에 추가
    #               path = folium.PolyLine(
    #                   locations=route_coordinates,
    #                   weight=3,
    #                   color='red',
    #                   opacity=0.8
    #               ).add_to(m)

    #               # 경로의 중간 지점 계산
    #               mid_point_index = len(route_coordinates) // 2
    #               mid_point = route_coordinates[mid_point_index]

    #               # 거리 정보를 표시하는 팝업 추가
    #               folium.Popup(
    #                   f'거리: {distance_km:.1f}km',
    #                   permanent=True
    #               ).add_to(folium.RegularPolygonMarker(
    #                   location=mid_point,
    #                   number_of_sides=4,
    #                   radius=0,
    #                   weight=0,
    #                   fill=False
    #               ).add_to(m))

    #       except Exception as e:
    #           print(f"경로 생성 중 오류 발생: {e}")
    #           continue

    #   return m
    # def get_route(self, start_lat, start_lng, dest_lat, dest_lng):
    #     url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    #     headers = {
    #         "X-NCP-APIGW-API-KEY-ID": 'sb7vih3g35',
    #         "X-NCP-APIGW-API-KEY": '4hp1aMpvQ9mRmhQ6h9U1xd5hV96x2so6Iz74vYiU',
    #     }
    #     params = {
    #         "start": f"{start_lng},{start_lat}",
    #         "goal": f"{dest_lng},{dest_lat}",
    #         "option": "trafast"
    #     }
    #     response = requests.get(url, headers=headers, params=params)
    #     if response.status_code == 200:
    #         data = response.json()
    #         try:
    #             # 경로 데이터 가져오기
    #             route = data['route']['trafast'][0]['path']
    #             return polyline.encode(route)  # 경로를 폴리라인으로 인코딩
    #         except (KeyError, IndexError):
    #             raise ValueError("올바르지 않은 응답 데이터 형식입니다.")
    #     else:
    #         raise ConnectionError(f"API 요청 실패: {response.status_code}, {response.text}")
    def calculate_road_distance(self, start_coords, end_coords):
    """
    실제 도로를 따라가는 경로의 거리를 계산합니다.

    Args:
        start_coords: (위도, 경도) 형태의 시작점 좌표
        end_coords: (위도, 경도) 형태의 도착점 좌표

    Returns:
        distance_km: 도로 경로를 따라가는 거리 (킬로미터)
        duration_min: 예상 소요 시간 (분)
    """
        try:
            # OSRM API 호출
            url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}"
            response = requests.get(url)

            if response.status_code == 200:
                data = response.json()
                if data['code'] == 'Ok':
                    # 거리를 미터에서 킬로미터로 변환
                    distance_km = data['routes'][0]['distance'] / 1000
                    # 시간을 초에서 분으로 변환
                    duration_min = data['routes'][0]['duration'] / 60

                    return distance_km, duration_min

            return None, None
        except Exception as e:
            print(f"거리 계산 중 오류 발생: {e}")
            return None, None

    def visualize_hospitals_with_route(self, user_coords, hospitals):
        # 사용자 위치를 중심으로 지도를 생성
        m = folium.Map(location=user_coords, zoom_start=13)

        # 사용자 위치 마커 추가
        folium.Marker(
            location=user_coords,
            tooltip="사용자 위치",
            icon=folium.Icon(color="blue")
        ).add_to(m)

        # 병원 위치 마커 추가 (추천 병원들)
        for _, row in hospitals.iterrows():
            # 실제 도로 경로에 따른 거리 계산
            hospital_coords = (row['위도'], row['경도'])
            road_distance, duration = self.calculate_road_distance(user_coords, hospital_coords)

            # 병원 마커 추가
            if road_distance is not None:
                hospital_info = f"{row['name']}\n도로 거리: {road_distance:.1f}km\n예상 소요시간: {duration:.0f}분"
            else:
                hospital_info = f"{row['name']}\n거리 계산 불가"

            folium.Marker(
                location=hospital_coords,
                popup=hospital_info,
                tooltip=row['name'],
                icon=folium.Icon(color="red")
            ).add_to(m)

            # 경로 표시 추가
            try:
                # OSRM API를 사용하여 경로 데이터 가져오기
                url = f"http://router.project-osrm.org/route/v1/driving/{user_coords[1]},{user_coords[0]};{row['경도']},{row['위도']}?overview=full&geometries=geojson"
                response = requests.get(url)

                if response.status_code == 200:
                    route_data = response.json()

                    # GeoJSON 형식의 경로 데이터를 추출
                    route_coordinates = route_data['routes'][0]['geometry']['coordinates']

                    # 좌표 순서 변환 (경도,위도 -> 위도,경도)
                    route_coordinates = [[coord[1], coord[0]] for coord in route_coordinates]

                    # 경로를 지도에 추가
                    path = folium.PolyLine(
                        locations=route_coordinates,
                        weight=3,
                        color='red',
                        opacity=0.8,
                        tooltip=f'거리: {road_distance:.1f}km'
                    ).add_to(m)

            except Exception as e:
                print(f"경로 생성 중 오류 발생: {e}")
                continue

        return m

if __name__ == "__main__":
  path = '/content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/'
  save_directory = path + 'fine_tuned_bert_v2'
  audio_path = path + "audio/"
  hospital_data = path + "응급실 정보.csv"
  openai.api_key = load_file(path + 'api_key.txt')
  os.environ['OPENAI_API_KEY'] = openai.api_key

  assistant = EmergencyAssistant(save_directory, audio_path, hospital_data)

  # Audio 파일 처리
  results = assistant.process_audio_files()
  for filename, text, summary, predicted_class in results:
      print(f"파일: {filename}, 요약: {summary}, 분류: {predicted_class}")

      if predicted_class <= 2:  # 응급환자
          address = input("응급환자 입니다. 주소를 입력해주세요: ")

          recommendations = pd.DataFrame(assistant.handle_emergency(address))
          print("추천 병원:")
          for idx, row in recommendations.iterrows():
              print(f"-----------------------------------")
              print(f"병원 이름    : {row['name']}")
              print(f"주소        : {row['address']}")
              print(f"거리 (m)    : {row['distance']} m")
              print(f"-----------------------------------\n")

          # 병원 시각화
          lat, lng = assistant.get_lat_long(address)
          user_coords = (lat, lng)
          hospital_map = assistant.visualize_hospitals_with_route(user_coords, recommendations)
          # hospital_map.save("hospital_map.html")

          # 지도를 노트북에서 바로 보기
          display(hospital_map)  # 주피터 노트북 환경에서 지도를 바로 렌더링

          print("병원 위치가 포함된 지도를 화면에 표시합니다.")

      else:
          print("응급환자가 아닙니다.")


      break



Overwriting /content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/emergency.py


In [ ]:
path = '/content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/'
save_directory = path + 'fine_tuned_bert_v2'
audio_path = path + "audio/"
hospital_data = path + "응급실 정보.csv"
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

assistant = EmergencyAssistant(save_directory, audio_path, hospital_data)

# Audio 파일 처리
results = assistant.process_audio_files()
for filename, text, summary, predicted_class in results:
    print(f"파일: {filename}, 요약: {summary}, 분류: {predicted_class}")

    if predicted_class <= 2:  # 응급환자
        address = input("응급환자 입니다. 주소를 입력해주세요: ")

        recommendations = pd.DataFrame(assistant.handle_emergency(address))
        print("추천 병원:")
        for idx, row in recommendations.iterrows():
            print(f"-----------------------------------")
            print(f"병원 이름    : {row['name']}")
            print(f"주소        : {row['address']}")
            print(f"거리 (m)    : {row['distance']} m")
            print(f"-----------------------------------\n")

        # 병원 시각화
        lat, lng = assistant.get_lat_long(address)
        user_coords = (lat, lng)
        hospital_map = assistant.visualize_hospitals_with_route(user_coords, recommendations)
        # hospital_map.save("hospital_map.html")

        # 지도를 노트북에서 바로 보기
        display(hospital_map)  # 주피터 노트북 환경에서 지도를 바로 렌더링

        print("병원 위치가 포함된 지도를 화면에 표시합니다.")

    else:
        print("응급환자가 아닙니다.")


    break


파일: audio1.mp3, 요약: {"summary": "아빠가 계단에서 넘어져서 머리에서 피가 나고 어지럽다고 하며 숨을 쉬고 있습니다. 응급실로 이송하는 것이 중요하며 신속한 의료 조치가 필요합니다."}, 분류: 0
응급환자 입니다. 주소를 입력해주세요: 송파구 올림픽공원
추천 병원:
-----------------------------------
병원 이름    : 성심의료재단강동성심병원
주소        : 서울특별시 강동구 성안로 150 (길동)
거리 (m)    : 2624 m
-----------------------------------

-----------------------------------
병원 이름    : 경찰병원
주소        : 서울특별시 송파구 송이로 123, 국립경찰병원 (가락동)
거리 (m)    : 3131 m
-----------------------------------

-----------------------------------
병원 이름    : 한국보훈복지의료공단중앙보훈병원
주소        : 서울특별시 강동구 진황도로61길 53 (둔촌동)
거리 (m)    : 3220 m
-----------------------------------



병원 위치가 포함된 지도를 화면에 표시합니다.


In [5]:
!pip install polyline


In [9]:
# %%writefile '/content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/emergency.py'


import os
import requests
import pandas as pd
import torch
import folium
import polyline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from geopy.geocoders import Nominatim
from openai import OpenAI
from warnings import filterwarnings

filterwarnings('ignore')

class EmergencyAssistant:
    def __init__(self, save_directory, audio_path, hospital_data):
        self.save_directory = save_directory
        self.audio_path = audio_path
        self.hospital_data = pd.read_csv(hospital_data)
        self.client = OpenAI()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = AutoModelForSequenceClassification.from_pretrained(save_directory).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(save_directory)
        # Naver API 키 설정
        self.naver_client_id = 'sb7vih3g35'
        self.naver_client_secret = '4hp1aMpvQ9mRmhQ6h9U1xd5hV96x2so6Iz74vYiU'

    def audio_to_text(self, filename):
        audio_file = open(os.path.join(self.audio_path, filename), "rb")
        transcript = self.client.audio.transcriptions.create(
            file=audio_file,
            model="whisper-1",
            language="ko",
            response_format="text",
        )
        return transcript

    def text_summary(self, input_text):
        system_role = '''당신은 신문기사에서 핵심을 요약하는 어시스턴트입니다.
        응답은 다음의 형식을 지켜주세요:
        {"summary": "텍스트 요약"}
        '''
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": input_text},
            ]
        )
        return response.choices[0].message.content

    def predict(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits.softmax(dim=1)
        predicted_class = torch.argmax(logits, dim=-1).item()
        return predicted_class, logits

    def get_lat_long(self, address):
        geolocator = Nominatim(user_agent="emergency_assistant")
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            raise ValueError("주소를 찾을 수 없습니다.")

    def get_dist(self, start_lat, start_lng, dest_lat, dest_lng):
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.naver_client_id,
            "X-NCP-APIGW-API-KEY": self.naver_client_secret
        }
        params = {
            "start": f"{start_lng},{start_lat}",
            "goal": f"{dest_lng},{dest_lat}",
            "option": "trafast"
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            try:
                return data['route']['trafast'][0]['summary']['distance']
            except (KeyError, IndexError):
                raise ValueError("올바르지 않은 응답 데이터 형식입니다.")
        else:
            raise ConnectionError(f"API 요청 실패: {response.status_code}, {response.text}")

    def process_audio_files(self):
        file_names = [f for f in os.listdir(self.audio_path) if os.path.isfile(os.path.join(self.audio_path, f))]
        results = []
        for filename in file_names:
            text = self.audio_to_text(filename)
            summary = self.text_summary(text)
            predicted_class, _ = self.predict(summary)
            results.append((filename, text, summary, predicted_class))
        return results

    def handle_emergency(self, address):
        lat, lng = self.get_lat_long(address)
        recommendations = self.recommend_hospital(lat, lng)
        return recommendations

    def calculate_road_distance(self, start_coords, end_coords):
        """
        네이버 API를 사용하여 실제 도로를 따라가는 경로의 거리와 시간을 계산합니다.
        """
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.naver_client_id,
            "X-NCP-APIGW-API-KEY": self.naver_client_secret
        }
        params = {
            "start": f"{start_coords[1]},{start_coords[0]}",  # 경도,위도 순서
            "goal": f"{end_coords[1]},{end_coords[0]}",
            "option": "trafast"
        }

        try:
            response = requests.get(url, headers=headers, params=params)
            if response.status_code == 200:
                data = response.json()
                route = data['route']['trafast'][0]
                # 거리를 미터에서 킬로미터로 변환
                distance_km = route['summary']['distance'] / 1000
                # 시간을 초에서 분으로 변환
                duration_min = route['summary']['duration'] / 60000  # 밀리초를 분으로 변환
                # 경로 좌표 추출
                path_coordinates = route['path']

                return distance_km, duration_min, path_coordinates
            return None, None, None
        except Exception as e:
            print(f"거리 계산 중 오류 발생: {e}")
            return None, None, None

    def visualize_hospitals_with_route(self, user_coords, hospitals):
        # 사용자 위치를 중심으로 지도를 생성
        m = folium.Map(location=user_coords, zoom_start=13)

        # 사용자 위치 마커 추가
        folium.Marker(
            location=user_coords,
            tooltip="사용자 위치",
            icon=folium.Icon(color="blue")
        ).add_to(m)

        # 병원 위치 마커 추가 (추천 병원들)
        for _, row in hospitals.iterrows():
            # 실제 도로 경로에 따른 거리 계산
            hospital_coords = (row['위도'], row['경도'])
            road_distance, duration, path_coordinates = self.calculate_road_distance(user_coords, hospital_coords)

            # 병원 마커 추가
            if road_distance is not None:
                hospital_info = (
                    f"<div style='font-size: 12px;'>"
                    f"<b>{row['name']}</b><br>"
                    f"도로 거리: {road_distance:.1f}km<br>"
                    f"예상 소요시간: {duration:.0f}분"
                    f"</div>"
                )
            else:
                hospital_info = f"{row['name']}\n거리 계산 불가"

            folium.Marker(
                location=hospital_coords,
                popup=folium.Popup(hospital_info, max_width=200),
                tooltip=row['name'],
                icon=folium.Icon(color="red")
            ).add_to(m)

            # 경로 표시 추가
            if path_coordinates:
                # 네이버 API에서 받은 좌표를 folium 형식으로 변환
                route_coordinates = [[coord[1], coord[0]] for coord in path_coordinates]

                # 경로를 지도에 추가
                folium.PolyLine(
                    locations=route_coordinates,
                    weight=3,
                    color='red',
                    opacity=0.8,
                    tooltip=f'거리: {road_distance:.1f}km, 소요시간: {duration:.0f}분'
                ).add_to(m)

                # 경로의 중간 지점에 거리 정보 표시
                mid_point_index = len(route_coordinates) // 2
                mid_point = route_coordinates[mid_point_index]

                folium.Popup(
                    f'거리: {road_distance:.1f}km<br>소요시간: {duration:.0f}분',
                    max_width=200
                ).add_to(folium.RegularPolygonMarker(
                    location=mid_point,
                    number_of_sides=4,
                    radius=0,
                    weight=0,
                    fill=False
                ).add_to(m))

        return m

    def recommend_hospital(self, lat, lng, alpha=0.1):
        """
        주변 병원을 추천하고 실제 도로 거리를 계산합니다.
        """
        target_hospitals = self.hospital_data[
            (self.hospital_data['위도'] >= lat - alpha) &
            (self.hospital_data['위도'] <= lat + alpha) &
            (self.hospital_data['경도'] >= lng - alpha) &
            (self.hospital_data['경도'] <= lng + alpha)
        ]

        recommendations = []
        for _, hospital in target_hospitals.iterrows():
            distance, duration, _ = self.calculate_road_distance(
                (lat, lng),
                (hospital['위도'], hospital['경도'])
            )
            if distance is not None:
                recommendations.append({
                    'name': hospital['병원이름'],
                    'address': hospital['주소'],
                    '위도': hospital['위도'],
                    '경도': hospital['경도'],
                    'distance': distance * 1000,  # km를 m로 변환
                    'duration': duration
                })

        # 거리순으로 정렬
        recommendations.sort(key=lambda x: x['distance'])
        return pd.DataFrame(recommendations).head(3)

    def save_map(self, map_obj, filename="hospital_routes.html"):
        """
        지도를 HTML 파일로 저장합니다.
        """
        map_obj.save(filename)


if __name__ == "__main__":
    # 경로 설정
    path = '/content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/'
    save_directory = path + 'fine_tuned_bert_v2'
    audio_path = path + "audio/"
    hospital_data = path + "응급실 정보.csv"

    # OpenAI API 키 설정
    def load_file(filepath):
        with open(filepath, 'r') as file:
            return file.read().strip()

    openai.api_key = load_file(path + 'api_key.txt')
    os.environ['OPENAI_API_KEY'] = openai.api_key

    # EmergencyAssistant 인스턴스 생성
    assistant = EmergencyAssistant(save_directory, audio_path, hospital_data)

    try:
        # Audio 파일 처리
        print("음성 파일 처리를 시작합니다...")
        results = assistant.process_audio_files()

        for filename, text, summary, predicted_class in results:
            print("\n" + "="*50)
            print(f"파일명: {filename}")
            print(f"텍스트 요약: {summary}")
            print(f"응급도 분류: {predicted_class}")
            print("="*50 + "\n")

            if predicted_class <= 2:  # 응급환자
                print("⚠️ 응급환자로 분류되었습니다!")

                # 주소 입력 및 예외 처리
                while True:
                    try:
                        address = input("\n🏥 현재 위치 주소를 입력해주세요: ")
                        recommendations = assistant.handle_emergency(address)
                        break
                    except ValueError as e:
                        print(f"\n❌ 오류: {e}")
                        print("주소를 다시 입력해주세요.")
                    except Exception as e:
                        print(f"\n❌ 예상치 못한 오류가 발생했습니다: {e}")
                        print("주소를 다시 입력해주세요.")

                # 추천 병원 정보 출력
                print("\n📍 추천 병원 목록:")
                for idx, row in recommendations.iterrows():
                    print("\n" + "-"*50)
                    print(f"🏥 병원 이름    : {row['name']}")
                    print(f"📍 주소        : {row['address']}")
                    print(f"🚗 이동 거리    : {row['distance']:.0f}m")
                    print(f"⏱️ 예상 소요시간 : {row['duration']:.0f}분")
                    print("-"*50)

                # 지도 시각화
                try:
                    print("\n🗺️ 지도를 생성하는 중입니다...")
                    lat, lng = assistant.get_lat_long(address)
                    user_coords = (lat, lng)
                    hospital_map = assistant.visualize_hospitals_with_route(user_coords, recommendations)

                    # 지도 저장
                    map_filename = "hospital_routes.html"
                    hospital_map.save(map_filename)
                    print(f"\n✅ 지도가 '{map_filename}'로 저장되었습니다.")

                    # 주피터 노트북 환경에서 지도 표시
                    try:
                        from IPython.display import display
                        display(hospital_map)
                    except ImportError:
                        print("\n📝 지도 파일을 웹 브라우저에서 열어주세요.")

                except Exception as e:
                    print(f"\n❌ 지도 생성 중 오류가 발생했습니다: {e}")

            else:
                print("ℹ️ 응급환자가 아닌 것으로 분류되었습니다.")
                print("일반 진료를 권장드립니다.")

            print("\n다음 음성 파일 처리를 시작합니다...\n")

    except Exception as e:
        print(f"\n❌ 프로그램 실행 중 오류가 발생했습니다: {e}")

    finally:
        print("\n프로그램을 종료합니다.")

음성 파일 처리를 시작합니다...

파일명: audio1.mp3
텍스트 요약: {"summary": "아빠가 계단에서 넘어져 머리에서 피가 나고 숨소리가 어간 들리며 일어난 상황. 현재 응급실로 이동할 필요가 있으며, 도움을 요청하는 것이 중요하다. 상황을 냉정하게 판단하고 신속히 대처하는 것이 중요하다."}
응급도 분류: 0

⚠️ 응급환자로 분류되었습니다!

🏥 현재 위치 주소를 입력해주세요: 대전 탄방빌딩

❌ 오류: 주소를 찾을 수 없습니다.
주소를 다시 입력해주세요.

🏥 현재 위치 주소를 입력해주세요: 대전 탄방역

📍 추천 병원 목록:

--------------------------------------------------
🏥 병원 이름    : 학교법인을지학원대전을지대학교병원
📍 주소        : 대전광역시 서구 둔산서로 95 (둔산동)
🚗 이동 거리    : 1962m
⏱️ 예상 소요시간 : 7분
--------------------------------------------------

--------------------------------------------------
🏥 병원 이름    : 학교법인가톨릭학원가톨릭대학교대전성모병원
📍 주소        : 대전광역시 중구 대흥로 64 (대흥동)
🚗 이동 거리    : 4597m
⏱️ 예상 소요시간 : 17분
--------------------------------------------------

--------------------------------------------------
🏥 병원 이름    : 충남대학교병원
📍 주소        : 대전광역시 중구 문화로 282 (대사동, 충남대학교병원)
🚗 이동 거리    : 4650m
⏱️ 예상 소요시간 : 16분
--------------------------------------------------

🗺️ 지도를 생성하는 중입니다...

✅ 지도가 'hospital_routes.html'로 저장되었습니다.



다음 음성 파일 처리를 시작합니다...


파일명: audio4.mp3
텍스트 요약: {"summary": "머리를 박으면서 어지러움과 메스꺼움을 느끼고 있는데 3시간이 지났다면 긴급 의료 지원이 필요합니다. 즉시 의사나 응급실로 연락하여 상황을 설명하고 조언을 받아야 합니다."}
응급도 분류: 0

⚠️ 응급환자로 분류되었습니다!

프로그램을 종료합니다.


KeyboardInterrupt: Interrupted by user

In [ ]:
%%writefile /content/drive/MyDrive/project6_2/emergency.py

import os
import requests
import pandas as pd
import openai
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from dotenv import load_dotenv
import folium


# 0. Load keys from environment ------------------
def load_env_keys(filepath="/content/drive/MyDrive/project6_2/maps.env"):
    load_dotenv(filepath)
    return os.getenv("NAVER_MAPS_CLIENT_ID"), os.getenv("NAVER_MAPS_CLIENT_SECRET"), os.getenv("OPENAI_API_KEY"), os.getenv("VWORLD_API_KEY")


# 키 등록 함수
def set_openai_api_key(api_key):
    openai.api_key = api_key


# 환경 변수 로드 및 키 설정
naver_client_id, naver_client_secret, openai_api_key, vworld_api_key = load_env_keys()
set_openai_api_key(openai_api_key)


# 1-1. Audio to Text ------------------
def audio_to_text(audio_path, filename):
    audio_file = open(os.path.join(audio_path, filename), "rb")
    response = openai.Audio.transcribe(
        model="whisper-1",
        file=audio_file,
        language="ko",
        response_format="text",
        temperature=0.0
    )
    return response['text']  # 텍스트 데이터 반환


# 1-2. Text to Summary ------------------
def text_to_summary(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Summarize the given emergency situation in a concise manner."},
            {"role": "user", "content": text}
        ],
        temperature=0.7
    )
    return response['choices'][0]['message']['content']


# 2. Emergency Level Prediction ------------------
def load_emergency_model(model_path="/content/drive/MyDrive/project6_2/fine_tuned_bert"):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    return tokenizer, model


def predict_emergency_level(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    probabilities = outputs.logits.softmax(dim=-1)
    predicted_level = torch.argmax(probabilities, dim=-1).item() + 1
    return predicted_level


# 3-1. Geocoding API: Convert Address to Coordinates ------------------
def get_coord_from_address(address):
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": naver_client_id,
        "X-NCP-APIGW-API-KEY": naver_client_secret,
    }
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['addresses']:
            location = data['addresses'][0]
            return float(location['y']), float(location['x'])  # 위도(y), 경도(x)
    print(f"Geocoding 실패: 주소={address}")
    return None


# 3-2. Distance Calculation ------------------
def get_dist(user_coords, hospital_coords):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": naver_client_id,
        "X-NCP-APIGW-API-KEY": naver_client_secret,
    }
    params = {
        "start": f"{user_coords[1]},{user_coords[0]}",
        "goal": f"{hospital_coords[1]},{hospital_coords[0]}",
        "option": "trafast"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['code'] == 0:
            return data['route']['trafast'][0]['summary']['distance'] / 1000  # Convert to km
    print(f"경로 탐색 실패 (병원: {hospital_coords})")
    return float('inf')  # 경로를 찾지 못하면 무한대 반환


# 3-3. Recommend Hospitals and Visualize with VWorld ------------------
def recommend_and_visualize_hospitals(lat, lng, df, vworld_api_key, n=3, alpha=0.1):
    # 병원 필터링
    filtered_df = df[
        (df['위도'] >= lat - alpha) &
        (df['위도'] <= lat + alpha) &
        (df['경도'] >= lng - alpha) &
        (df['경도'] <= lng + alpha)
    ]
    print(f"필터링된 병원 수: {len(filtered_df)}")

    # 거리 계산
    distances = []
    for _, row in filtered_df.iterrows():
        dest_coords = (row['위도'], row['경도'])
        distance = get_dist((lat, lng), dest_coords)
        distances.append(distance)

    # 거리 정보 추가
    filtered_df = filtered_df.copy()
    filtered_df['거리(km)'] = distances

    # 가까운 병원 정렬 후 상위 n개 추천
    recommended_hospitals = filtered_df.sort_values(by='거리(km)').head(n)

    # 지도 생성
    m = folium.Map(location=(lat, lng), zoom_start=12)

    # VWorld 지도 타일 추가
    folium.TileLayer(
        tiles=f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_api_key}/Base/{{z}}/{{y}}/{{x}}.png",
        attr="Tiles © VWorld",
        name="VWorld Base",
        overlay=False
    ).add_to(m)
    folium.TileLayer(
        tiles=f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_api_key}/Satellite/{{z}}/{{y}}/{{x}}.jpeg",
        attr="Tiles © VWorld",
        name="VWorld Satellite",
        overlay=False
    ).add_to(m)
    folium.TileLayer(
        tiles=f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_api_key}/Hybrid/{{z}}/{{y}}/{{x}}.png",
        attr="Tiles © VWorld",
        name="VWorld Hybrid",
        overlay=False
    ).add_to(m)

    folium.Marker(
        location=(lat, lng),
        tooltip="사용자 위치",
        icon=folium.Icon(color="blue")
    ).add_to(m)

    # 병원 정보 시각화
    for _, row in recommended_hospitals.iterrows():
        hospital_coords = (row['위도'], row['경도'])
        hospital_name = row['병원이름']
        distance = row['거리(km)']

        # 구글맵 URL 생성
        google_maps_url = f"https://www.google.com/maps?q={hospital_coords[0]},{hospital_coords[1]}"

        # 팝업 HTML 생성
        popup_html = f"""
        <b>{hospital_name}</b><br>
        거리: {distance:.2f} km<br>
        <a href="{google_maps_url}" target="_blank" style="color:blue; text-decoration:underline;">
        구글맵에서 위치 보기</a>
        """
        popup = folium.Popup(popup_html, max_width=300)

        # 병원 마커 추가
        folium.Marker(
            location=hospital_coords,
            popup=popup,
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(m)

        # 사용자와 병원 사이 경로 표시
        folium.PolyLine(
            [(lat, lng), hospital_coords],
            color="red",
            weight=2.5,
            tooltip=f"거리: {distance:.2f} km"
        ).add_to(m)

    # 지도 전환 컨트롤 추가
    folium.LayerControl().add_to(m)

    return m, recommended_hospitals


In [3]:
import os
import openai
import requests
import folium
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

class EmergencyResponseSystem:
    def __init__(
        self,
        naver_client_id,
        naver_client_secret,
        vworld_api_key,
        model_save_directory,
        audio_path,
        hospital_data
    ):
        """
        Initialize the emergency response system.

        Args:
            naver_client_id (str): Naver Maps API client ID
            naver_client_secret (str): Naver Maps API client secret
            vworld_api_key (str): VWorld API key
            model_save_directory (str): Path to the fine-tuned BERT model directory
            audio_path (str): Path to audio files directory
            hospital_data (str): Path to hospital data CSV file
        """
        # API 키 설정
        self.naver_client_id = naver_client_id
        self.naver_client_secret = naver_client_secret
        self.vworld_api_key = vworld_api_key

        # 경로 설정
        self.model_path = model_save_directory
        self.audio_path = audio_path

        # 병원 데이터 로드
        self.hospital_df = pd.read_csv(hospital_data)

        # 응급 상황 분류 모델 로드
        self.tokenizer, self.model = self._load_emergency_model()

    def _load_emergency_model(self):
        """Load the emergency classification model."""
        tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        model = AutoModelForSequenceClassification.from_pretrained(self.model_path)
        return tokenizer, model

    def process_audio(self, filename):
        """Convert audio to text using OpenAI's Whisper model."""
        audio_file = open(os.path.join(self.audio_path, filename), "rb")
        response = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file,
            language="ko",
            response_format="text",
            temperature=0.0
        )
        return response

    def summarize_text(self, text):
        """Summarize emergency situation text using GPT-3.5."""
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Summarize the given emergency situation in a concise manner."},
                {"role": "user", "content": text}
            ],
            temperature=0.7
        )
        return response['choices'][0]['message']['content']

    def predict_emergency_level(self, text):
        """Predict emergency level using the fine-tuned BERT model."""
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        outputs = self.model(**inputs)
        probabilities = outputs.logits.softmax(dim=-1)
        return torch.argmax(probabilities, dim=-1).item() + 1

    def get_coordinates(self, address):
        """Convert address to coordinates using Naver Geocoding API."""
        url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.naver_client_id,
            "X-NCP-APIGW-API-KEY": self.naver_client_secret,
        }
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            if data['addresses']:
                location = data['addresses'][0]
                return float(location['y']), float(location['x'])
        print(f"Geocoding 실패: 주소={address}")
        return None

    def calculate_distance(self, user_coords, hospital_coords):
        """Calculate distance between two points using Naver Direction API."""
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.naver_client_id,
            "X-NCP-APIGW-API-KEY": self.naver_client_secret,
        }
        params = {
            "start": f"{user_coords[1]},{user_coords[0]}",
            "goal": f"{hospital_coords[1]},{hospital_coords[0]}",
            "option": "trafast"
        }
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            if data['code'] == 0:
                return data['route']['trafast'][0]['summary']['distance'] / 1000
        print(f"경로 탐색 실패 (병원: {hospital_coords})")
        return float('inf')



    def find_and_visualize_hospitals(self, lat, lng, n=3, alpha=0.1):
        """Find nearest hospitals and create visualization."""
        # 병원 필터링
        filtered_df = self.hospital_df[
            (self.hospital_df['위도'] >= lat - alpha) &
            (self.hospital_df['위도'] <= lat + alpha) &
            (self.hospital_df['경도'] >= lng - alpha) &
            (self.hospital_df['경도'] <= lng + alpha)
        ]
        print(f"필터링된 병원 수: {len(filtered_df)}")

        # 거리 계산
        filtered_df = filtered_df.copy()
        filtered_df['거리(km)'] = [
            self.calculate_distance((lat, lng), (row['위도'], row['경도']))
            for _, row in filtered_df.iterrows()
        ]

        # 추천 병원 선정
        recommended_hospitals = filtered_df.sort_values(by='거리(km)').head(n)

        # 지도 생성
        m = folium.Map(location=(lat, lng), zoom_start=12)

        # VWorld 타일 추가
        for layer_type in ['Base', 'Satellite', 'Hybrid']:
            url = f"http://api.vworld.kr/req/wmts/1.0.0/{self.vworld_api_key}/{layer_type}/{{z}}/{{y}}/{{x}}.{'png' if layer_type != 'Satellite' else 'jpeg'}"
            folium.TileLayer(
                tiles=url,
                attr="Tiles © VWorld",
                name=f"VWorld {layer_type}",
                overlay=False
            ).add_to(m)

        # 사용자 위치 마커
        folium.Marker(
            location=(lat, lng),
            tooltip="사용자 위치",
            icon=folium.Icon(color="blue")
        ).add_to(m)

        # 병원 마커 및 경로 추가
        for _, row in recommended_hospitals.iterrows():
            hospital_coords = (row['위도'], row['경도'])
            google_maps_url = f"https://www.google.com/maps?q={hospital_coords[0]},{hospital_coords[1]}"

            popup_html = f"""
            <b>{row['병원이름']}</b><br>
            거리: {row['거리(km)']:.2f} km<br>
            <a href="{google_maps_url}" target="_blank" style="color:blue; text-decoration:underline;">
            구글맵에서 위치 보기</a>
            """

            folium.Marker(
                location=hospital_coords,
                popup=folium.Popup(popup_html, max_width=300),
                icon=folium.Icon(color="red", icon="info-sign")
            ).add_to(m)

            folium.PolyLine(
                [(lat, lng), hospital_coords],
                color="red",
                weight=2.5,
                tooltip=f"거리: {row['거리(km)']:.2f} km"
            ).add_to(m)

        folium.LayerControl().add_to(m)
        return m, recommended_hospitals

In [6]:
path = '/content/drive/MyDrive/kt/projects_kt/project_6_2/6_2/'
save_directory = path + 'fine_tuned_bert_v2'
audio_path = path + "audio/"
hospital_data = path + "응급실 정보.csv"
def load_file(filepath):
        with open(filepath, 'r') as file:
            return file.read().strip()
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key
ers = EmergencyResponseSystem(
    naver_client_id="sb7vih3g35",
    naver_client_secret="4hp1aMpvQ9mRmhQ6h9U1xd5hV96x2so6Iz74vYiU",
    vworld_api_key=" A4632994-A7F1-35F3-918C-565CC2345896",
    model_save_directory=save_directory,
    audio_path = audio_path,
    hospital_data = hospital_data
)

# 오디오 처리
text = ers.process_audio("audio1.mp3")

# 텍스트 요약
summary = ers.summarize_text(text)

# 응급 수준 예측
level = ers.predict_emergency_level(text)

# 좌표로 병원 찾기
map_obj, hospitals = ers.find_and_visualize_hospitals(
    lat=37.5665,
    lng=126.9780
)



필터링된 병원 수: 50


In [7]:
def display_results(map_obj, hospitals):
    """
    지도와 병원 정보를 함께 표시하는 함수

    Args:
        map_obj (folium.Map): 생성된 지도 객체
        hospitals (pd.DataFrame): 추천된 병원 정보
    """
    from IPython.display import display, HTML

    # 1. 지도 표시
    display(map_obj)

    # 2. 병원 정보 테이블 스타일 정의
    html_style = """
    <style>
        .hospital-table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 20px;
            font-family: Arial, sans-serif;
        }
        .hospital-table th, .hospital-table td {
            padding: 12px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }
        .hospital-table th {
            background-color: #f5f5f5;
            font-weight: bold;
            color: #333;
        }
        .hospital-table tr:hover {
            background-color: #f9f9f9;
        }
        .distance {
            color: #e74c3c;
            font-weight: bold;
        }
    </style>
    """

    # 3. 병원 정보 테이블 HTML 생성
    table_html = """
    <table class="hospital-table">
        <tr>
            <th>순번</th>
            <th>병원명</th>
            <th>거리</th>
            <th>위도</th>
            <th>경도</th>
        </tr>
    """

    for idx, (_, hospital) in enumerate(hospitals.iterrows(), 1):
        table_html += f"""
        <tr>
            <td>{idx}</td>
            <td>{hospital['병원이름']}</td>
            <td class="distance">{hospital['거리(km)']:.2f} km</td>
            <td>{hospital['위도']}</td>
            <td>{hospital['경도']}</td>
        </tr>
        """

    table_html += "</table>"

    # 4. 스타일과 테이블 표시
    display(HTML(html_style + table_html))

# 사용 예시:
# display_results(map_obj, hospitals)

In [8]:
display_results(map_obj, hospitals)


순번,병원명,거리,위도,경도
1,강북삼성병원,1.31 km,37.56849763123383,126.96793805451702
2,서울적십자병원,1.64 km,37.56715536263689,126.96699861289684
3,세란병원,3.03 km,37.57534016994642,126.9577071892358
